In [83]:
import numpy as np
import pandas as pd
import datetime
import time
import re
import talib
import operator
import psycopg2
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.header import Header
import matplotlib.pyplot as plt

%matplotlib inline

# All those that need to be changed are here:

In [85]:
location =  "C:/Users/workspace/brad_public_workspace_on_win/SH_tongliang/strats/CTA_strats/11_22_pairs_trading/signals/"
tail_str = 'u18z18'
starttime = '2018-06-29 00:00:00'
endtime = '2018-12-02 00:00:00'
# acttime = '08:05:00'
receiver = ['bradsun91@163.com']   
# Gary's: 304381005@qq.com'
number_of_assts = 6

frequency = "_1d"
file_suffix = 'u18z18' + frequency
time = "12/2 morning signal: "

# 手动更新实时价格：
ADA_live_price = 0.00000983
ETH_live_price = 0.02880
BCH_live_price = 0.0443
LTC_live_price = 0.00809
EOS_live_price = 0.0007706
TRX_live_price = 0.00000352

ADA_leverage = 1  
ETH_leverage = 1  
BCH_leverage = 1  
LTC_leverage = 1  
EOS_leverage = 1  
TRX_leverage = 1  

# 定义资金和杠杆规模：
total_capital = 0.39         # 单位：XBT
capital_utility = 0.80/number_of_assts    # 每个策略不超过的总资金百分比, e.g. total_capital的%再除以number_of_assts

# Signal Generation

In [86]:
"""
自动数据库读取数据，计算排序，发送邮件
"""

# 计算两天close涨幅 T1,T2
def zf(data):
    resdata = []
    for r in data.itertuples():
        time = getattr(r, 'time')
        open = getattr(r, 'open')
        high = getattr(r, 'high')
        low = getattr(r, 'low')
        close = getattr(r, 'close')
        resdata.append([time, open, high, low, close])
    res = [0]
    for i in range(1, len(resdata)):
        T = (float(resdata[i][4]) - float(resdata[i - 1][4])) / float(resdata[i - 1][4]) * 100
        res.append(T)
    return res


# 计算TP和值
def myTP(T1, T2, Q):
    res = [0]

    for i in range(len(T1)):
        t = T1[i] * (1 - Q / 100) + T2[i] * (Q / 100)
        res.append(t)
    return res


# 计算TP差值
def myTM(T1, T2, Q):
    res = [0]
    for i in range(len(T1)):
        t = T1[i] * (1 - Q / 100) - T2[i] * (Q / 100)
        res.append(t)
    return res


# 计算和值累加 或 差值累加
def mySUM(TP):
    res = [0]
    for i in TP:
        res.append(res[-1] + i)
    return res


# 求指定周期均值，参数1数据列表，参数2周期，返回单个数值
def myAverage(lists, tp):
    if len(lists) > tp:
        return np.mean(lists[-tp:])
    else:
        return np.mean(lists)


# 判断趋势
def pdqs(lists):
    count = -1
    while lists[count] == lists[count - 1]:
        count -= 1
    if lists[count] > lists[count - 1]:
        return "趋势向上"
    elif lists[count] < lists[count - 1]:
        return "趋势向下"


# 判断交叉
# def pdjx(ddlist, eelist):
#     for i in range(len(ddlist)):
#         if ddlist[-i]:
#             return "金叉"
#         elif eelist[-i]:
#             return "死叉"
#         else:
#             return '没有交叉'
# 判断交叉
def pdjx(ddlist, eelist):
    for i in range(len(ddlist)):
        if ddlist[-i]:
            return "金叉"
        elif eelist[-i]:
            return "死叉"
        else:
            return '没有交叉'


# 计算结果 参数：数据1，数据2，短周期，长周期，权重1，权重2,截止日期
def comres(data0, data1, S, L, Q1, Q2, starttime, endtime):
    """计算两者排名"""
    pd.to_datetime(data0.ix[:, 0])  # 转换为时间格式
    pd.to_datetime(data1.ix[:, 0])  # 转换为时间格式
#     print ("comres process starttime: ", starttime)
#     print ("comres process endtime: ", endtime)
    # timestr = '2018-10-19 00:00:00'

    data0 = data0[(data0.ix[:, 0] <= endtime) & (data0.ix[:, 0] >= starttime)]
    data1 = data1[(data1.ix[:, 0] <= endtime) & (data1.ix[:, 0] >= starttime)]
#     print('筛选后')
#     print(len(data0), len(data1))
    T1 = zf(data0)  # 第一腿涨幅
#     print ("T1: ",T1)
    T2 = zf(data1)  # 第二腿涨幅
#     print ("T2: ",T2)

    TP = myTP(T1, T2, Q1)  # 相对和
    TM = myTM(T1, T2, Q2)  # 相对差

    TPSUM = mySUM(TP)  # 相对和累加
    TMSUM = mySUM(TM)  # 相对差累加

    TMS = np.array(TMSUM)
    TPS = np.array(TPSUM)

    MA1 = talib.MA(TMS, S)  # 相对差短周期均线
    MA2 = talib.MA(TMS, L)  # 相对差长周期均线
#     print ("MA1", MA1)
#     print ("MA2", MA2)
    # MAS = talib.MA(TPS, S)
    # MAL = talib.MA(TPS, L)

    if MA1[-1] > MA2[-1]:
        jx = '金叉' # ada increases faster
    else:
        jx = '死叉' # eth increases faster

    MA1qs = pdqs(MA1)
    MA2qs = pdqs(MA2)
    # print(MA2)
    # print("交叉：" + jx)
    # print("短周期相对差累加均线MA1趋势:" + MA1qs)
    # print("长周期相对差累加均线MA2趋势:" + MA2qs)
    if jx == '金叉':
#         print ("ADA强于ETH，做空ADA, 做多ETH") 
        return 1
        
    if jx == '死叉':
#         print ("ADA弱于ETH，做多ADA, 做空ETH")
        return -1
        

def get_data(conn, sym, period, tail_str):
    """获取数据, tail_str = e.g. 'u18z18' """
    if sym == 'BTC':
        sym = 'xbtusd'
    else:
        sym = sym.lower() + tail_str
    sqllasttime = " SELECT time, open, high, low, close, volume from " + sym + "_" + period + " order by time"
    df = pd.read_sql(sqllasttime, con=conn)
    return df



def send_email(username, password, receiver, text):
    """发送邮件"""
    smtpserver = 'smtp.qq.com'
    sender = username

    subject = '强弱排名'
    subject = Header(subject, 'utf-8').encode()

    msg = MIMEMultipart('mixed')
    msg['Subject'] = subject
    msg['From'] = username
    # msg['To'] = 'XXX@126.com'
    # 收件人为多个收件人,通过join将列表转换为以;为间隔的字符串
    msg['To'] = ";".join(receiver)
    # msg['Date']='2012-3-16'

    text_plain = MIMEText(text, 'plain', 'utf-8')
    msg.attach(text_plain)

    # 发送邮件
    smtp = smtplib.SMTP()
    smtp.connect('smtp.qq.com')
    # 我们用set_debuglevel(1)就可以打印出和SMTP服务器交互的所有信息。
    # smtp.set_debuglevel(1)
    smtp.login(username, password)
    smtp.sendmail(sender, receiver, msg.as_string())
    smtp.quit()


def sleeptime(timestr):
    now_time = datetime.datetime.now()
    nowhms = time.strftime("%H:%M:%S")
    # 目标时间与今天还未到达
    if nowhms < timestr:
        now_year = now_time.year
        now_month = now_time.month
        now_day = now_time.day
        aimtime = datetime.datetime.strptime(str(now_year) + "-" + str(now_month) + "-" + str(now_day) + " " + timestr,
                                             "%Y-%m-%d %H:%M:%S")
        sleeptime = (aimtime - now_time).total_seconds()
        print('当前时间', now_time)
        print('目标时间', aimtime)
        print('睡眠时间', sleeptime)
        time.sleep(sleeptime)
    else:
        # 目标时间今天已过
        aimtime = now_time + datetime.timedelta(days=+1)
        aim_year = aimtime.year
        aim_month = aimtime.month
        aim_day = aimtime.day
        aimtime = datetime.datetime.strptime(str(aim_year) + "-" + str(aim_month) + "-" + str(aim_day) + " " + timestr,
                                             "%Y-%m-%d %H:%M:%S")
        sleeptime = (aimtime - now_time).total_seconds()
        print('当前时间', now_time)
        print('目标时间', aimtime)
        print('睡眠时间', sleeptime)
        time.sleep(sleeptime)
        
        
        
        
        
location = location
tail_str = tail_str
starttime = starttime
endtime = endtime


if __name__ == '__main__':
    ########################################
#     first_endtime = '2018-06-29 00:00:00'
#     first_endtime = '2018-11-20 00:00:00'
#     break_time = '2018-11-25 00:00:00'
    ########################################
    conn = psycopg2.connect(database="bitmexdata", user="postgres", password="tongKen123", host="128.199.97.202", port="5432")
    signal_record = {"日期": [],
                     "ADA方向": [],
                     "ETH方向": [],
                     "BCH方向":[],
                     "LTC方向":[],
                     "EOS方向":[],
                     "TRX方向":[]}
    
#     acttime = acttime
#     sleeptime(acttime)
#     if break_time < first_endtime:
#         break
    username = '499428970@qq.com'
    password = 'oklvhkylrbtqbjja'
    sender = '499428970@qq.com'
    # 收件人为多个收件人
#     receiver = ['bradsun91@163.com', '304381005@qq.com']
    receiver = receiver             

    # 合约列表
    symlist = ['ADA', 'ETH', 'BCH', 'LTC', 'EOS', 'TRX']
    binSize = '1d'
    ####################################
#         starttime = '2018-06-29 00:00:00'
    starttime = starttime
    endtime = endtime
    ####################################
    

    Q1 = 50  # 权重1
    Q2 = 50  # 权重2 相对差
    
    # Pair 1 parameter:
    S01 = 5  # 短周期
    L01 = 26  # 长周期
    
    # Pair 2 parameter:
    S23 = 10
    L23 = 11
    
    # Pair 3 parameter:
    S45 = 6
    L45 = 7

    sym0 = symlist[0]
    sym1 = symlist[1]
    sym2 = symlist[2]
    sym3 = symlist[3]
    sym4 = symlist[4]
    sym5 = symlist[5]

#     tail_str = tail_str = 'u18z18'
    data0 = get_data(conn, sym0, binSize, tail_str)
    data1 = get_data(conn, sym1, binSize, tail_str)
    data2 = get_data(conn, sym2, binSize, tail_str)
    data3 = get_data(conn, sym3, binSize, tail_str)
    data4 = get_data(conn, sym4, binSize, tail_str)
    data5 = get_data(conn, sym5, binSize, tail_str)


    res01 = comres(data0, data1, S01, L01, Q1, Q2, starttime, endtime)
    res23 = comres(data2, data3, S23, L23, Q1, Q2, starttime, endtime)
    res45 = comres(data4, data5, S45, L45, Q1, Q2, starttime, endtime)
#         print ("res:", res)

    endtime = datetime.datetime.strptime(endtime, '%Y-%m-%d %H:%M:%S')
    wendtime = str((endtime + datetime.timedelta(days=-1)).strftime('%Y-%m-%d'))
    ts_text = wendtime[:10] + ' 收盘时排名'

    # ADA and ETH
    if res01 ==1:
        signal_record['日期'].append(ts_text)      
        signal_record['ADA方向'].append(-1)
        signal_record['ETH方向'].append(1)

    else:
        signal_record['日期'].append(ts_text)      
        signal_record['ADA方向'].append(1)
        signal_record['ETH方向'].append(-1)         

    # BCH and LTC
    if res23 ==1:
#             signal_record['日期'].append(ts_text)      
        signal_record['BCH方向'].append(-1)
        signal_record['LTC方向'].append(1)

    else:
#             signal_record['日期'].append(ts_text)      
        signal_record['BCH方向'].append(1)
        signal_record['LTC方向'].append(-1)   


    # EOS and TRX
    if res45 ==1:
#             signal_record['日期'].append(ts_text)      
        signal_record['EOS方向'].append(-1)
        signal_record['TRX方向'].append(1)

    else:
#             signal_record['日期'].append(ts_text)      
        signal_record['EOS方向'].append(1)
        signal_record['TRX方向'].append(-1) 

    print ("{} data processed".format(wendtime[:10]))

#     first_endtime = str((endtime + datetime.timedelta(days=1)).strftime('%Y-%m-%d  %H:%M:%S'))
    signal_df = pd.DataFrame(signal_record)
#     signal_df.to_csv(location + "{}_brad_strat_signal_most_updated.csv".format(wendtime[:10]))
    a = np.array(list(signal_df.columns))
    b = np.array(list(signal_df.iloc[-1, :]))
    zipped = list(zip(a,b))
    text = ""
    for i in zipped:
        text = text+i[0]+':'+i[1]+'\n'
    #     print(text)    
    #     restext = wendtime[:10] + '排名\n'
    send_email(username, password, receiver, text)
    conn.close()

C:\Users\Brad Sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:91: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
C:\Users\Brad Sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:92: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
C:\Users\Brad Sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:97: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
C:\Users\Brad Sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:98: DeprecationWarning: 
.ix is deprec

2018-12-01 data processed


In [87]:
signal_df

,日期,ADA方向,ETH方向,BCH方向,LTC方向,EOS方向,TRX方向
0,2018-12-01 收盘时排名,1,-1,1,-1,1,-1


# Position Instruction

In [49]:
ada_signal = int(signal_df['ADA方向'].values)
eth_signal = int(signal_df['ETH方向'].values)
bch_signal = int(signal_df['BCH方向'].values)
ltc_signal = int(signal_df['LTC方向'].values)
eos_signal = int(signal_df['EOS方向'].values)
trx_signal = int(signal_df['TRX方向'].values)

frequency = frequency
file_suffix = file_suffix
time = time

def all_assts_from_sql(asst1, asst2, asst3, asst4, asst5, asst6, sql_limit_num):
    conn = psycopg2.connect(database="bitmexdata", user="postgres", password="tongKen123", host="128.199.97.202", port="5432")
    asset1 = "SELECT time, close from {} order by id desc limit {}".format(asst1, sql_limit_num)
    asset2 = "SELECT time, close from {} order by id desc limit {}".format(asst2, sql_limit_num)
    asset3 = "SELECT time, close from {} order by id desc limit {}".format(asst3, sql_limit_num)
    asset4 = "SELECT time, close from {} order by id desc limit {}".format(asst4, sql_limit_num)
    asset5 = "SELECT time, close from {} order by id desc limit {}".format(asst5, sql_limit_num)
    asset6 = "SELECT time, close from {} order by id desc limit {}".format(asst6, sql_limit_num)

    df1 = pd.read_sql(asset1, con=conn)
    df2 = pd.read_sql(asset2, con=conn)
    df3 = pd.read_sql(asset3, con=conn)
    df4 = pd.read_sql(asset4, con=conn)
    df5 = pd.read_sql(asset5, con=conn)
    df6 = pd.read_sql(asset6, con=conn)
    conn.close()
    return df1, df2, df3, df4, df5, df6


ada, bch, eos, eth, ltc, trx = all_assts_from_sql("ada"+file_suffix, 
                                                  "bch"+file_suffix, 
                                                  "eos"+file_suffix,
                                                  "eth"+file_suffix, 
                                                  "ltc"+file_suffix, 
                                                  "trx"+file_suffix,
                                                   1000)

ada.sort_values('time', inplace=True)
eth.sort_values('time', inplace=True)
bch.sort_values('time', inplace=True)
ltc.sort_values('time', inplace=True)
eos.sort_values('time', inplace=True)
trx.sort_values('time', inplace=True)

ada_pct = ada['close'].pct_change()
eth_pct = eth['close'].pct_change()
bch_pct = bch['close'].pct_change()
ltc_pct = ltc['close'].pct_change()
eos_pct = eos['close'].pct_change()
trx_pct = trx['close'].pct_change()

ada_max_vol = ada_pct.max()
eth_max_vol = eth_pct.max()
bch_max_vol = bch_pct.max()
ltc_max_vol = ltc_pct.max()
eos_max_vol = eos_pct.max()
trx_max_vol = trx_pct.max()

# print ("ADA 最大杠杆不应超过： ", 1/ada_max_vol, "倍")
# print ("ETH 最大杠杆不应超过： ", 1/eth_max_vol, "倍")
# print ("BCH 最大杠杆不应超过： ", 1/bch_max_vol, "倍")
# print ("LTC 最大杠杆不应超过： ", 1/ltc_max_vol, "倍")
# print ("EOS 最大杠杆不应超过： ", 1/eos_max_vol, "倍")
# print ("TRX 最大杠杆不应超过： ", 1/trx_max_vol, "倍")

# 手动更新实时价格：
ADA_live_price = ADA_live_price
ETH_live_price = ETH_live_price
BCH_live_price = BCH_live_price
LTC_live_price = LTC_live_price
EOS_live_price = EOS_live_price
TRX_live_price = TRX_live_price

ADA_signal = ada_signal
ETH_signal = eth_signal
BCH_signal = bch_signal
LTC_signal = ltc_signal
EOS_signal = eos_signal
TRX_signal = trx_signal

ADA_leverage = ADA_leverage  
ETH_leverage = ETH_leverage
BCH_leverage = BCH_leverage
LTC_leverage = LTC_leverage  
EOS_leverage = EOS_leverage  
TRX_leverage = TRX_leverage  

# 定义资金和杠杆规模：
total_capital = total_capital         # 单位：XBT
capital_utility = capital_utility    # 每个策略不超过的总资金百分比 %
            # 杠杆率
# position_number = 6       # 目标资产数量 

def execution_instruction(ADA_live_price, ADA_signal, ADA_leverage,
                          ETH_live_price, ETH_signal, ETH_leverage,
                          BCH_live_price, BCH_signal, BCH_leverage,
                          LTC_live_price, LTC_signal, LTC_leverage,
                          EOS_live_price, EOS_signal, EOS_leverage,
                          TRX_live_price, TRX_signal, TRX_leverage,
                          total_capital, capital_utility):
    each_allocation = total_capital*capital_utility
    ada_size = ADA_signal*each_allocation/ADA_live_price*ADA_leverage
    eth_size = ETH_signal*each_allocation/ETH_live_price*ETH_leverage
    bch_size = BCH_signal*each_allocation/BCH_live_price*BCH_leverage
    ltc_size = LTC_signal*each_allocation/LTC_live_price*LTC_leverage
    eos_size = EOS_signal*each_allocation/EOS_live_price*EOS_leverage
    trx_size = TRX_signal*each_allocation/TRX_live_price*TRX_leverage
    
    return ada_size, eth_size, bch_size, ltc_size, eos_size, trx_size


ada_size, eth_size, bch_size, ltc_size, eos_size, trx_size = execution_instruction(ADA_live_price, ADA_signal, ADA_leverage,
                                                                                  ETH_live_price, ETH_signal, ETH_leverage,
                                                                                  BCH_live_price, BCH_signal, BCH_leverage,
                                                                                  LTC_live_price, LTC_signal, LTC_leverage,
                                                                                  EOS_live_price, EOS_signal, EOS_leverage,
                                                                                  TRX_live_price, TRX_signal, TRX_leverage,
                                                                                  total_capital, capital_utility)

# 观察资金数量是否足够至少买一个合约

compare_size = {}
compare_size["ada"] = ada_size
compare_size["eth"] = eth_size
compare_size["bch"] = bch_size
compare_size["ltc"] = ltc_size
compare_size["eos"] = eos_size
compare_size["trx"] = trx_size

live_prices = {}
live_prices["ada"] = ADA_live_price
live_prices["eth"] = ETH_live_price
live_prices["bch"] = BCH_live_price
live_prices["ltc"] = LTC_live_price
live_prices["eos"] = EOS_live_price
live_prices["trx"] = TRX_live_price

min_ = min(abs(ada_size), abs(eth_size), abs(bch_size), abs(ltc_size), abs(eos_size), abs(trx_size))

for k, v in compare_size.items():
#     print (k, v)
    if v == min_ and abs(v) < 1:
        
        print ("发现最贵目标仓位: {}, {:.2f}个, 开不出1整个合约，因此：".format(k, v))
        print ("==================================================")
        print ("将此最贵目标仓位升至1个合约数...")
        v_sign = v*abs(v)
        v = v_sign*v
        compare_size[k] = 1
        print ("==================================================")
        print ("最贵目标仓位设定完毕: {}, {}个".format(k, 1))
        print ("==================================================")
        print ("最贵目标仓位{}的对应总价值: {}".format(k, live_prices[k]))
#         print ("==================================================")
#         print ("因此其他每个仓位至少分配{}的倍数".format(live_prices[k]))
        print ("==================================================")
        print ("按最小公倍数，总需交易资金为: {}".format(live_prices[k]*number_of_assts))
        print ("==================================================")
        print ("账户总资金为： {}个BTC".format(total_capital))
        print ("==================================================")
        if total_capital >= number_of_assts*live_prices[k]:
            print ("账户总资产足够交易此{}个资产的策略".format(number_of_assts))
            print ("==================================================")
            each_capital = live_prices[k]
#             print ("发现最贵目标仓位: {}, {:.2f}个, 开得出至少1个合约".format(k, v))
            ada_new_size = live_prices[k]/live_prices['ada']
            eth_new_size = live_prices[k]/live_prices['eth']
            bch_new_size = live_prices[k]/live_prices['bch']
            ltc_new_size = live_prices[k]/live_prices['ltc']
            eos_new_size = live_prices[k]/live_prices['eos']
            trx_new_size = live_prices[k]/live_prices['trx']
#             print ("still runnnig")
            print ("==================================================")
            print ("交易指令: "+time,"ADA:",int(abs(ada_new_size))*ADA_signal, ", 单位：个", "价值：", int(ada_new_size)*live_prices['ada'])
            print ("交易指令: "+time,"ETH:",int(abs(eth_new_size))*ETH_signal, ", 单位：个", "价值：", int(eth_new_size)*live_prices['eth'])
            print ("==================================================")
            print ("交易指令: "+time,"BCH:",int(abs(bch_new_size))*BCH_signal, ", 单位：个", "价值：", int(bch_new_size)*live_prices['bch'])
            print ("交易指令: "+time,"LTC:",int(abs(ltc_new_size))*LTC_signal, ", 单位：个", "价值：", int(ltc_new_size)*live_prices['ltc'])
            print ("==================================================")
            print ("交易指令: "+time,"EOS:",int(abs(eos_new_size))*EOS_signal, ", 单位：个", "价值：", int(eos_new_size)*live_prices['eos'])
            print ("交易指令: "+time,"TRX:",int(abs(trx_new_size))*EOS_signal, ", 单位：个", "价值：", int(trx_new_size)*live_prices['trx'])
            
        else:
            print ("总资产不足够交易此策略")
        ada_sz = ada_new_size
        eth_sz = eth_new_size
        bch_sz = bch_new_size
        ltc_sz = ltc_new_size
        eos_sz = eos_new_size
        trx_sz = trx_new_size
            
    elif v == min_ and abs(v) >= 1:
        print ("==================================================")
        print ("发现最贵目标仓位: {}, {:.2f}个, 开得出至少1个合约".format(k, v))
        print ("==================================================")
        print ("交易指令: "+time,"ADA:",int(abs(ada_size))*ADA_signal, ", 单位：个", "价值：", int(ada_size)*live_prices['ada'])
        print ("交易指令: "+time,"ETH:",int(abs(eth_size))*ETH_signal, ", 单位：个", "价值：", int(eth_size)*live_prices['eth'])
        print ("==================================================")
        print ("交易指令: "+time,"BCH:",int(abs(bch_size))*BCH_signal, ", 单位：个", "价值：", int(bch_size)*live_prices['bch'])
        print ("交易指令: "+time,"LTC:",int(abs(ltc_size))*LTC_signal, ", 单位：个", "价值：", int(ltc_size)*live_prices['ltc'])
        print ("==================================================")
        print ("交易指令: "+time,"EOS:",int(abs(eos_size))*EOS_signal, ", 单位：个", "价值：", int(eos_size)*live_prices['eos'])
        print ("交易指令: "+time,"TRX:",int(abs(trx_size))*EOS_signal, ", 单位：个", "价值：", int(trx_size)*live_prices['trx'])
        
        ada_sz = ada_size
        eth_sz = eth_size
        bch_sz = bch_size
        ltc_sz = ltc_size
        eos_sz = eos_size
        trx_sz = trx_size

发现最贵目标仓位: bch, 1.17个, 开得出至少1个合约
交易指令: 12/1 morning signal:  ADA: 5289 , 单位：个 价值： 0.05199087
交易指令: 12/1 morning signal:  ETH: -1 , 单位：个 价值： -0.0288
交易指令: 12/1 morning signal:  BCH: 1 , 单位：个 价值： 0.0443
交易指令: 12/1 morning signal:  LTC: -6 , 单位：个 价值： -0.04854
交易指令: 12/1 morning signal:  EOS: 67 , 单位：个 价值： 0.0516302
交易指令: 12/1 morning signal:  TRX: 14772 , 单位：个 价值： -0.051997440000000006


If necessary, adjust to balanced positions using the function below

In [52]:
def adjust_two_legs(sml_asst, large_asst, sml_btc_num, mltplr, large_unit_prc):
    sml_val_trgt =  sml_btc_num*mltplr
    large_btc_num = int(sml_val_trgt/large_unit_prc)
    large_val_trgt = large_btc_num*large_unit_prc
    print ("{}调整为{}个BTC; ".format(sml_asst, mltplr), "价值(BTC): ", sml_val_trgt)
    print ("{}调整为{}个BTC; ".format(large_asst, large_btc_num), "价值(BTC): ", large_val_trgt)
    return sml_val_trgt, large_val_trgt

In [54]:
adjstd_sml_asst, adjstd_large_asst = adjust_two_legs("ETH", "ADA", 0.0288, 2, ADA_live_price)

ETH调整为2个BTC;  价值(BTC):  0.0576
ADA调整为5859个BTC;  价值(BTC):  0.05759397


In [67]:
# 验证新调整后的总交易资金小于总账户资金：
# 先重新赋值调整过的资产交易仓位：

ada_sz = adjstd_large_asst
eth_sz = adjstd_sml_asst
bch_sz = bch_sz*live_prices['bch']
ltc_sz = ltc_sz*live_prices['ltc']
eos_sz = eos_sz*live_prices['eos']
trx_sz = trx_sz*live_prices['trx']

In [72]:
all_size = abs(ada_sz)+abs(eth_sz)+abs(bch_sz)+abs(ltc_sz)+abs(eos_sz)+abs(trx_sz)
all_size

0.32319397

In [82]:
print ("新调整总交易资本利用率: ", round(all_size/total_capital*100, 2), "%")

新调整总交易资本利用率:  82.87 %


# Market Analysis

In [58]:
print ("ADA", data0[['close']].iloc[-1:, :])
print ("===============================================")
print ("ETH", data1[['close']].iloc[-1:, :])
print ("===============================================")
print ("BCH", data2[['close']].iloc[-1:, :])
print ("===============================================")
print ("LTC", data3[['close']].iloc[-1:, :])
print ("===============================================")
print ("EOS", data4[['close']].iloc[-1:, :])
print ("===============================================")
print ("TRX", data5[['close']].iloc[-1:, :])

ADA        close
163  0.00001
ETH        close
163  0.02758
BCH       close
163  0.0425
LTC        close
163  0.00785
EOS         close
163  0.000703
TRX         close
155  0.000004


In [59]:
print ("ADA", data0[['close']].pct_change().iloc[-2:, :])
print ("===============================================")
print ("ETH", data1[['close']].pct_change().iloc[-2:, :])
print ("===============================================")
print ("BCH", data2[['close']].pct_change().iloc[-2:, :])
print ("===============================================")
print ("LTC", data3[['close']].pct_change().iloc[-2:, :])
print ("===============================================")
print ("EOS", data4[['close']].pct_change().iloc[-2:, :])
print ("===============================================")
print ("TRX", data5[['close']].pct_change().iloc[-2:, :])

ADA         close
162  0.049370
163 -0.017017
ETH         close
162  0.006932
163 -0.050602
BCH         close
162 -0.034188
163 -0.059735
LTC         close
162  0.003699
163 -0.035627
EOS         close
162 -0.016137
163 -0.092212
TRX         close
154  0.115016
155  0.042980


In [7]:
signal_df

,日期,ADA方向,ETH方向,BCH方向,LTC方向,EOS方向,TRX方向
0,2018-11-30 收盘时排名,1,-1,1,-1,1,-1
